In [1]:
# Step 1: Install required libraries
!pip install -q langchain langchain-community langchain-google-genai google-generativeai langchain_chroma langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 944.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.6 MB/s eta 0:0

In [2]:
# Step 2: Import necessary modules
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    PromptTemplate,
    AIMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
import os
import getpass

In [3]:
# Step 3: Set up your Google API key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [4]:
# Step 4: Initialize the Gemini model
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")  # Use 'gemini-pro' as the model

In [5]:
examples = [
    {
        "question": "How to get from Isfahan to Tehran?",
        "answer": """
        option 1: mode=bus, min_time_in_min=360 max_time_in_min=390 description=Take bus from Kaveh Terminal.
        option 2: mode=plane min_time_in_min=60 max_time_in_min=70 description=Take plane from Iran Air, Mahan Air or Zagros Air.
        option 3: mode=taxi min_time_in_min=330 max_time_in_min=360 description=Take private taxi using Snapp.
        """
    },
    {
        "question": "How to get from Shiraz to Isfahan?",
        "answer": """
        option 1: mode=bus, min_time_in_min=360 max_time_in_min=390 description=Take bus from Karandish Terminal.
        option 2: mode=plane min_time_in_min=75 max_time_in_min=90 description=Take plane from Iran Air.
        option 3: mode=taxi min_time_in_min=320 max_time_in_min=350 description=Take private taxi using Snapp.
        """
    },
    {
        "question": "How to get from Tehran to Rasht?",
        "answer": """
        option 1: mode=bus, min_time_in_min=250 max_time_in_min=300 description=Take bus from Argentina Terminal.
        option 2: mode=plane min_time_in_min=45 max_time_in_min=55 description=Take plane from Iran Air.
        option 3: mode=taxi min_time_in_min=200 max_time_in_min=240 description=Take private taxi using Snapp.
        """
    },
]

In [6]:
##Example Selector

to_vectorize = [" ".join(example.values()) for example in examples]

#embedding
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

#vector store
vector_store = Chroma.from_texts(texts=to_vectorize, embedding=embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k=2
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
example_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("{question}"),
    AIMessagePromptTemplate.from_template("{answer}")
])

In [8]:
fewshot_prompt_template = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    input_variables=["question"]
)

In [9]:
system_prompt_template = PromptTemplate(
    template = "You are a travel expert helping to find efficient and comfortable way of travelling from point A to point B."
)

system_message_template = SystemMessagePromptTemplate(prompt=system_prompt_template)

In [10]:
user_prompt_template = PromptTemplate(
    template = "how to reach {target} from {source}?",
    input_variables=["target", "source"]
)

user_message_template = HumanMessagePromptTemplate(prompt=user_prompt_template)

In [11]:
chat_prompt_template_with_fewshot = ChatPromptTemplate.from_messages([
    system_message_template,
    fewshot_prompt_template,
    user_message_template
])

In [12]:
user_input_source = "Tehran" # @param{type: "string"}

In [13]:
user_input_target = "Yazd" # @param{type: "string"}

In [14]:
messages = chat_prompt_template_with_fewshot.format_messages(target=user_input_target, source=user_input_source)

In [15]:
#Print formatted messages
for msg in messages:
  print(msg)

content='You are a travel expert helping to find efficient and comfortable way of travelling from point A to point B.' additional_kwargs={} response_metadata={}
content='How to get from Isfahan to Tehran?' additional_kwargs={} response_metadata={}
content='\n        option 1: mode=bus, min_time_in_min=360 max_time_in_min=390 description=Take bus from Kaveh Terminal.\n        option 2: mode=plane min_time_in_min=60 max_time_in_min=70 description=Take plane from Iran Air, Mahan Air or Zagros Air.\n        option 3: mode=taxi min_time_in_min=330 max_time_in_min=360 description=Take private taxi using Snapp.\n        ' additional_kwargs={} response_metadata={}
content='How to get from Shiraz to Isfahan?' additional_kwargs={} response_metadata={}
content='\n        option 1: mode=bus, min_time_in_min=360 max_time_in_min=390 description=Take bus from Karandish Terminal.\n        option 2: mode=plane min_time_in_min=75 max_time_in_min=90 description=Take plane from Iran Air.\n        option 3

In [16]:
chat_model_response= chat_model.invoke(messages)

In [17]:
parser = StrOutputParser()
response = parser.invoke(chat_model_response)

In [18]:
print(response)

You have several options to travel from Tehran to Yazd:

* **Train:**  This is a popular and comfortable option.  Trains are generally overnight, departing Tehran in the evening and arriving in Yazd early morning. The journey takes about 6-8 hours.  Consider booking in advance, especially during peak season.
* **Bus:** Buses are more frequent and often cheaper than trains, but the journey is longer, taking around 7-9 hours.  VIP buses offer more comfortable seating.
* **Plane:**  Flying is the fastest option, taking around 1 hour.  Yazd Airport (AZD) is located a short distance from the city.  Consider the total travel time, including airport transfers, when comparing with other options.
* **Private Car/Taxi:** This is the most expensive option but offers flexibility and door-to-door service.  The drive takes approximately 6-7 hours.  You can hire a private driver or use a ride-hailing app like Snapp.

**Recommendation:** For comfort and a reasonable travel time, the overnight train is